In [1]:
import json
import pandas as pd
from collections import defaultdict, Counter

# === INPUT / OUTPUT ===
INPUT_OPENALEX_CSV = "openalex_works_full.csv"   # e.g., "/mnt/data/openalex_works_full.csv"
OUTPUT_PEOPLE_CSV = "openalex_people.csv"

RAW_COL = "raw_json"       # column that contains the OpenAlex work JSON
WORK_ID_COL = "openalex_id"

# === LOAD ===
df = pd.read_csv(INPUT_OPENALEX_CSV)
print(f"Works (rows) in input CSV: {len(df):,}")
print(f"Columns: {list(df.columns)}")

# === PARSE + AGGREGATE PEOPLE ===
people = {}  # author_id -> base info
author_insts = defaultdict(set)     # author_id -> set of institution names
author_work_ids = defaultdict(set)  # author_id -> set of work ids (from your input)
stats = Counter()

for _, row in df.iterrows():
    work_id = row.get(WORK_ID_COL)
    raw = row.get(RAW_COL)

    if pd.isna(raw) or not str(raw).strip():
        stats["works_missing_raw_json"] += 1
        continue

    try:
        w = json.loads(raw)
    except Exception:
        stats["works_bad_json"] += 1
        continue

    stats["works_parsed"] += 1

    authorships = w.get("authorships") or []
    stats["authorships_total"] += len(authorships)

    for a in authorships:
        author = a.get("author") or {}
        author_id = author.get("id")
        if not author_id:
            stats["authorships_missing_author_id"] += 1
            continue

        display_name = author.get("display_name")
        orcid = author.get("orcid")

        # initialize / update author info
        if author_id not in people:
            people[author_id] = {
                "openalex_id": author_id,
                "display_name_or_alias": display_name,  # OpenAlex works usually provide only display_name here
                "orcid": orcid,
            }
        else:
            if not people[author_id].get("display_name_or_alias") and display_name:
                people[author_id]["display_name_or_alias"] = display_name
            if not people[author_id].get("orcid") and orcid:
                people[author_id]["orcid"] = orcid

        if orcid:
            stats["authorships_with_orcid"] += 1

        # link author to work
        if work_id:
            author_work_ids[author_id].add(work_id)

        # institutions from this authorship
        for inst in (a.get("institutions") or []):
            inst_name = (inst.get("display_name") or "").strip()
            if inst_name:
                author_insts[author_id].add(inst_name)
                stats["institution_mentions"] += 1

# === BUILD OUTPUT DF ===
rows = []
for author_id, info in people.items():
    inst_list = sorted(author_insts.get(author_id, set()))
    rows.append({
        "openalex_id": info.get("openalex_id"),
        "display_name_or_alias": info.get("display_name_or_alias"),
        "orcid": info.get("orcid"),
        "institutions": "; ".join(inst_list),
        "n_institutions": len(inst_list),
        "n_works_in_input": len(author_work_ids.get(author_id, set())),
    })

people_df = pd.DataFrame(rows)

# sort: authors with more works first
people_df = people_df.sort_values(
    by=["n_works_in_input", "display_name_or_alias"],
    ascending=[False, True],
    na_position="last"
)

people_df.to_csv(OUTPUT_PEOPLE_CSV, index=False)

# === USEFUL COUNTS ===
n_people = len(people_df)
n_with_orcid = people_df["orcid"].notna().sum() if n_people else 0
n_unique_insts = len({i for s in author_insts.values() for i in s})

print("\n=== SUMMARY ===")
print(f"Works parsed (valid raw_json): {stats['works_parsed']:,}")
print(f"Works missing raw_json: {stats['works_missing_raw_json']:,}")
print(f"Works with invalid JSON in raw_json: {stats['works_bad_json']:,}")
print(f"Total authorships: {stats['authorships_total']:,}")
print(f"Authorships missing author.id: {stats['authorships_missing_author_id']:,}")

print("\n=== PEOPLE ===")
print(f"Unique authors extracted: {n_people:,}")
print(f"Authors with ORCID: {n_with_orcid:,} ({(n_with_orcid/n_people*100 if n_people else 0):.1f}%)")
print(f"Unique institutions (from authorships): {n_unique_insts:,}")
print(f"Institution mentions (total occurrences): {stats['institution_mentions']:,}")

if n_people:
    print(f"Avg institutions per author: {people_df['n_institutions'].mean():.2f}")
    print(f"Avg works per author (in your input set): {people_df['n_works_in_input'].mean():.2f}")

print(f"\nSaved: {OUTPUT_PEOPLE_CSV} (rows: {len(people_df):,})")

# preview
people_df.head(10)


Works (rows) in input CSV: 236
Columns: ['input_doi', 'openalex_id', 'doi', 'title', 'publication_year', 'type', 'cited_by_count', 'journal', 'authors', 'institutions', 'raw_json']

=== SUMMARY ===
Works parsed (valid raw_json): 236
Works missing raw_json: 0
Works with invalid JSON in raw_json: 0
Total authorships: 1,456
Authorships missing author.id: 9

=== PEOPLE ===
Unique authors extracted: 892
Authors with ORCID: 715 (80.2%)
Unique institutions (from authorships): 175
Institution mentions (total occurrences): 1,514
Avg institutions per author: 1.14
Avg works per author (in your input set): 1.61

Saved: openalex_people.csv (rows: 892)


,openalex_id,display_name_or_alias,orcid,institutions,n_institutions,n_works_in_input
140,https://openalex.org/A5007218240,Massimo De Vittorio,https://orcid.org/0000-0003-1601-6392,Center for Biomolecular Nanotechnologies; Ecos...,8,22
364,https://openalex.org/A5028454417,Ferruccio Pisanello,https://orcid.org/0000-0002-1489-7758,Center for Biomolecular Nanotechnologies; Ecos...,6,17
331,https://openalex.org/A5043864991,Maura Casadio,https://orcid.org/0000-0003-2338-8995,University of Genoa,1,16
135,https://openalex.org/A5017047546,Liam Collard,https://orcid.org/0000-0002-9742-9043,Center for Biomolecular Nanotechnologies; Ecos...,6,12
136,https://openalex.org/A5092828505,Linda Piscopo,https://orcid.org/0000-0002-1976-1518,Center for Biomolecular Nanotechnologies; Ital...,3,11
362,https://openalex.org/A5072431247,Filippo Pisano,https://orcid.org/0000-0002-5499-160X,Center for Biomolecular Nanotechnologies; Ital...,3,10
1,https://openalex.org/A5079829556,Giuseppe Patanè,https://orcid.org/0000-0002-2276-9553,Institute of Intelligent Systems for Automatio...,3,10
273,https://openalex.org/A5055281228,Carlo S. Regazzoni,https://orcid.org/0000-0001-6617-1417,University of Genoa,1,9
169,https://openalex.org/A5025286783,Silvia Biasotti,https://orcid.org/0000-0002-9992-825X,Istituto di Matematica Applicata e Tecnologie ...,1,9
150,https://openalex.org/A5038381549,Chiara Romanengo,https://orcid.org/0000-0002-9459-6209,Istituto di Matematica Applicata e Tecnologie ...,1,8
